# Assignment 3 - DATA.ML.360

In [1]:
%pip install pandas
%pip install surprise

import pandas as pd

You should consider upgrading via the '/Users/laurira/uni/recsys/assignment3/a3_venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/laurira/uni/recsys/assignment3/a3_venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Read the data file and see how it looks like
df = pd.read_csv('u.data', sep='\t', header=None)

# Add column names and check few rows of the dataset
df.columns = ["user_id", "movie_id", "rating", "timestamp"]
df = df.drop("timestamp", axis=1)

df.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


### Singular Value Decomposition (SVD)
Matrix factorization
Explanation here, what and why

Steps:
- Train the model sequentially
- Create group recommendations for that sequence
- Lump the sequences recommendations together, get top 10 recommendations

Sources used:

https://gregorygundersen.com/blog/2018/12/10/svd/#:~:text=The%20singular%20values%20referred%20to,our%20transformation%20flattens%20our%20square.

https://towardsdatascience.com/how-to-build-a-movie-recommendation-system-67e321339109

https://www.math3ma.com/blog/understanding-entanglement-with-svd


In [3]:
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Create a dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'movie_id', 'rating']], reader)

# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Create the svd algorithm
svd = SVD()

# Train the algorithm on the training set
svd.fit(trainset)

# Check the accuracy of the test set
predictions = svd.test(testset)
accuracy.rmse(predictions)


RMSE: 0.9412


0.9411649149426561

In [4]:
def create_recommendations(user_id):
    # Get movies that user hasnt rated yet
    user_ratings = df[df['user_id'] == user_id]
    user_unrated_movies = df[~df['movie_id'].isin(user_ratings['movie_id'])]['movie_id'].unique()

    user_recommendations = []
    # Loop through the unrated movies
    for movie_id in user_unrated_movies:
        # Create a predicted score for the movie
        prediction = svd.predict(user_id, movie_id)
        # Add it to the recommendations
        user_recommendations.append((user_id, movie_id, prediction.est))

    # Retrun the top 10 best rated predictions
    user_recommendations = sorted(user_recommendations, key=lambda x: x[2], reverse=True)[:10]
    return user_recommendations

In [5]:
# Select a set of users
users = [1, 2, 3]
group_recommendations = []

# Go thorugh each user and create predictions.
for user_id in users:
    group_recommendations.extend(create_recommendations(user_id))

# Create a dataframe from the ratings
rating_df = pd.DataFrame(group_recommendations, columns=["user_id", "movie_id", "predicted_rating"])

display(rating_df)

,user_id,movie_id,predicted_rating
0,1,318,4.991978
1,1,483,4.817680
2,1,408,4.812137
3,1,923,4.696408
4,1,654,4.605307
5,1,705,4.599508
6,1,427,4.559659
7,1,480,4.483890
8,1,963,4.465550
9,1,750,4.464973
